In [ ]:
library(dplyr)
library(tibble)
library(DBI)
library(RMySQL)

rm(list=ls())

source(paste0("/data/repos/actin-personalization/scripts/personalization/find_similar_patients_functions.R"))

dbActinPersonalization <- dbConnect(MySQL(), dbname='actin_personalization', groups="RAnalysis")

query_ref <-"select e.*, d.ageAtDiagnosis, d.hasRasMutation, GROUP_CONCAT(locationGroup ORDER BY locationGroup) AS metastasisLocationGroupsPriorToSystemicTreatment from episode e join diagnosis d on e.diagnosisId=d.id JOIN metastasis mm ON mm.episodeId=e.id AND intervalTumorIncidenceMetastasisDetection < intervalTumorIncidenceTreatmentPlanStart GROUP BY e.id;"

ref <- dbGetQuery(dbActinPersonalization, query_ref)
dbDisconnect(dbActinPersonalization)

ref_general <- find_similar_patients_general(ref)
head(ref_general[, c("ageAtDiagnosis", "observedPfsDays", "hadProgressionEvent")])

In [ ]:
library(ggsurvfit)
survfit2(Surv(observedPfsDays, hadProgressionEvent) ~ 1, data = ref_general) %>% 
  ggsurvfit() +
  labs(
    x = "Days",
    y = "Overall survival probability"
  ) + 
  add_confidence_interval() +
  add_risktable()

In [ ]:
library(survminer)
age <- 50

ref_age <- find_similar_patients_age(ref_general, patient_age=age, range=5)
ref_who <- find_similar_patients_who(ref_general, patient_who=1)
ref_ras <- find_similar_patients_ras(ref_general, patient_ras_status=0)
ref_lesions <- find_similar_patients_lesions(ref_general, patient_lesions=c("Liver"))

ages <- ref_general[['ageAtDiagnosis']]
greatest_age_diff <- max(age - min(ages), max(ages) - age)
weights <- apply(ref_general, 1, function(row) 1 - abs(as.numeric(row[['ageAtDiagnosis']]) - age)/greatest_age_diff)

general_surv_fit <- survfit(Surv(observedPfsDays, hadProgressionEvent)~1, data=ref_general)
age_surv_fit <- survfit(Surv(observedPfsDays, hadProgressionEvent)~1, data=ref_age)
who_surv_fit <- survfit(Surv(observedPfsDays, hadProgressionEvent)~1, data=ref_who)
ras_surv_fit <- survfit(Surv(observedPfsDays, hadProgressionEvent)~1, data=ref_ras)
lesion_surv_fit <- survfit(Surv(observedPfsDays, hadProgressionEvent)~1, data=ref_lesions)
weighted_surv_fit <- survfit(Surv(observedPfsDays, hadProgressionEvent)~1, data=ref_general, weights=weights)
surv_fit_list <- list(
    "All" = general_surv_fit,
    "45-55" = age_surv_fit,
#    "WHO=1" = who_surv_fit,
#    "RAS WT" = ras_surv_fit,
#    "Lesions" = lesion_surv_fit,
    "Weighted" = weighted_surv_fit
)

ggsurvplot(
    surv_fit_list,
    combine=TRUE,
    xlim = c(0,1200),
    break.time.by = 200
)

In [ ]:
treatment_surv_fit <- survfit(Surv(observedPfsDays, hadProgressionEvent)~systemicTreatmentPlan, data=ref_general, weights=weights)
treatmentsurv <- ggsurvplot(
    treatment_surv_fit,
    xlim = c(0,1200),
    surv.median.line = "hv",
    legend = "top",
    conf.int = TRUE,
    break.time.by = 200
)
treatmentsurv$plot <- treatmentsurv$plot + 
    theme(legend.text = element_text(size = 8, color = "black"))
treatmentsurv

In [ ]:
treatment_surv_fit